In [ ]:
import requests
import json
from api_key import api_key
import pandas as pd
import time
import pickle

pd.options.display.max_rows = 9999

In [ ]:
class Stock():
    def __init__(self, symbol,api_key):
        self.symbol=symbol
        self.api_key=api_key
        self.api_output=self.get_api_data()
        self.weight=1
        
    def get_api_data(self):
        """make api call"""
        #Would want to handle non-200 responses 
        r=requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+self.symbol+'&outputsize=full&apikey='+self.api_key)
        return r
    
    def get_stock_df(self):
        """Create a dataframe indexed by day wwhich has share price info"""
        stock_hist=json.loads(self.api_output.content)
        stock_df=pd.DataFrame(stock_hist['Time Series (Daily)']).transpose()
        stock_df=stock_df.astype(float)
        stock_df['9. symbol']=stock_hist['Meta Data']['2. Symbol']+' '
        return stock_df
    

class Index():
    #Need to do the API calls in the index because it's limited to 5/minute and 500/day
    def __init__(self):
        self.sp500_df=self.get_sp_df()
        self.sp500_prices={}
        self.api_all_stocks()
    def get_sp_df(self):
        #Need a better source for SP500 financials like mkcp
        sp500=requests.get('https://datahub.io/core/s-and-p-500-companies-financials/r/constituents-financials.json')
        sp500_df=pd.DataFrame(json.loads(sp500.content))
        sp500_df['Shares']=sp500_df['Market Cap']/sp500_df['Price']
        return sp500_df
    def sample_sp(self,num_sample):
        return self.sp500_df.sample(num_sample)
    
    def api_all_stocks(self):
        symb_list=self.sp500_df['Symbol'].tolist()
        for i in symb_list:
            self.sp500_prices[i]=Stock(i, api_key)
            time.sleep(15)

    
    
class Portfolio(Index):
    def __init__(self, symbol_list):
        self.symbol_list=symbol_list
        self.Stocks=self.get_all_stocks()
    def get_all_stocks(self):
        Stocks=[]
        for symbol in self.symbol_list:
            Stocks.append(Stock(symbol,api_key))
        return Stocks
    def total_portfolio_df(self):
        loop_check=0
        for stock in self.Stocks:
            if loop_check==0:
                agg_df=stock.weight*stock.get_stock_df()
            else:
                agg_df+=stock.get_stock_df()
        

In [ ]:
pickle.dump( sp500, open( "2019-06-01_save.p", "wb" ))

In [ ]:
sp500 = pickle.load( open( "2019-06-01_save.p", "rb" ) )

In [ ]:
sp500.sp500_prices['XYL']=Stock('XYL',api_key)
sp500.sp500_prices['YUM']=Stock('YUM',api_key)
sp500.sp500_prices['ZBH']=Stock('ZBH',api_key)
sp500.sp500_prices['ZION']=Stock('ZION',api_key)
sp500.sp500_prices['ZTS']=Stock('ZTS',api_key)

In [ ]:
sp500.sp500_df

In [ ]:
def return_2018(symbol):
    shares=1000/float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)']['2018-01-02']['5. adjusted close'])
    return shares*float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)']['2018-12-31']['5. adjusted close'])

In [ ]:
pd.DataFrame(json.loads(sp500.sp500_prices['WYN'].api_output.content)['Time Series (Daily)']).transpose()

In [ ]:
class Performance():
    def __init__(self,num_stocks=0, stock_list=None,start_date='2018-01-02', end_date='2018-12-31'):
        self.num_stock=num_stocks
        self.start_date=start_date
        self.end_date=end_date
        
        if stock_list is None:
            self.stock_list=self.get_stock_list(num_stocks)
        else:
            self.stock_list=stock_list
        
        self.stock_weights=self.get_stock_weights(self.stock_list)
        
        
    def get_adj_price(self, symbol, date=None):
        if date is None:
            date=self.start_date
        try:
            return float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][date]['5. adjusted close'])
        except:
            return 0
    def get_nom_price(self,symbol,date=None):
        if date is None:
            date=self.start_date
        try: 
            return float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][date]['4. close'])
        except:
            return 0
        
    def get_mkt_cap(self,symbol,date=None):
        if date is None:
            date=self.start_date
        try:
            return sp500.sp500_df[sp500.sp500_df['Symbol']==symbol]['Shares'].tolist()[0]*z.get_nom_price(symbol,date)
        except:
            return 0
    
    def get_total_return(self,symbol,start_date, end_date,weight=1000.00):
        shares=weight/float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][start_date]['5. adjusted close'])
        return shares*float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][end_date]['5. adjusted close'])
    
    def get_stock_weights(self,stock_list):
        weights={}
        tot_cap=0
        for i in stock_list:
            tot_cap+=get_mkt_cap(i)
        for i in stock_list:
            weights[i]=get_mkt_cap(i)/tot_cap
        return weights
        
    def portfolio_return(self):
        port_return=0
        for i in self.stock_list:
            port_return+=self.get_total_return(i,self.start_date,self.end_date,self.stock_weights[i])
        return port_return
        
    def get_stock_list(self,num_stocks):
        stock_list=[]
        for i in range(num_stocks):
            symbol=sample_stock(stock_list)
            stock_list.append(symbol)
        return stock_list
    
    def sample_stock(self, stock_list):
        stock_sample=sp500.sp500_df.sample(1)['Symbol'].tolist()[0]
        if stock_sample not in stock_list:
            try: 
                test=json.loads(sp500.sp500_prices[stock_sample].api_output.content)['Time Series (Daily)'][self.start_date]
                test2=json.loads(sp500.sp500_prices[stock_sample].api_output.content)['Time Series (Daily)'][self.end_date]
                return stock_sample
            except:
                return sample_stock(stock_list)
        else:
            return sample_stock(stock_list)

In [ ]:
z=Performance(stock_list=sp500.sp500_df['Symbol'].tolist())



print(z.portfolio_return())
#print(sp500.sp500_df[sp500.sp500_df['Symbol']=='AMZN']['Shares'].tolist()[0]*z.get_nom_price('AMZN'))

In [ ]:
sp500.sp500_df['Symbol'].tolist()

In [ ]:
def random_perf(num_stocks):
    
    
    #stock_list

    total_return=0
    for i in stock_list:
        #print(i)
        total_return+=return_2018(i)
        
    return total_return

In [ ]:
performance_results=[]
for i in range (100):
    performance_results.append(random_perf(50))

In [ ]:
import statistics
print(max(performance_results))
print(min(performance_results))

print(statistics.mean(performance_results))
print(statistics.stdev(performance_results))

In [ ]:
10841.229755636177
7526.546772302236
9072.111642584901
621.0216097915476

10951.0677977124
7476.093964348738
9091.096682826419
663.6401236122267

In [ ]:
def avg_list(inputs):
    for i in inputs:
        total+=i
    return total/len(inputs)

In [ ]:
spy=Stock('SPY',api_key)

In [ ]:
spy_shares=1/float(json.loads(spy.api_output.content)['Time Series (Daily)']['2018-01-02']['5. adjusted close'])
spy_shares*float(json.loads(spy.api_output.content)['Time Series (Daily)']['2018-12-31']['5. adjusted close'])

In [ ]:
pd.DataFrame(json.loads(sp500.sp500_prices['DIS'].api_output.content)['Time Series (Daily)']).transpose()

In [ ]:
port=Portfolio(sample_stocks['Symbol'].tolist())

In [ ]:
print(port.Stocks[5].symbol,port.Stocks[5].api_output.content)

In [ ]:
port.total_portfolio_df()

In [ ]:
stock1=Stock('DIS',api_key)
stock2=Stock('T',api_key)
combined=stock1.get_stock_df().join(stock2.get_stock_df(),lsuffix=' '+stock1.symbol,rsuffix=' '+stock2.symbol)

In [ ]:
stock1.get_stock_df()+stock2.get_stock_df()

In [ ]:
#combined

In [ ]:
combined[['5. adjusted close '+stock1.symbol,'5. adjusted close '+stock2.symbol]].astype(float).corr()

In [ ]:


stock_list=sp500_df.sample(5)['Symbol'].tolist()
index=Portfolio(stock_list)
index.Stocks[0].get_stock_df()

In [ ]:
z=Index()

In [ ]:
z.

In [ ]:
z=Stock('DIS',api_key)
z.get_stock_df()